# How does SHA-256 work?
### Simple hasing algorithm 256

In [15]:
from hashlib import sha256

m = sha256()
print(f"Hash type:\t{m.name}")
print(f"Void digest:\t{m.hexdigest()}")
print(f"Digest size:\t{m.digest_size}")

string = "Hello World!"
m.update(string.encode())
print(f"\nString:\t{string}")
print(f"Digest:\t{m.hexdigest()}")


Hash type:	sha256
Void digest:	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Digest size:	32

String:	Hello World!
Digest:	7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Atomic bit level operations

1. SHR
2. ROTR
3. XOR
4. SUM

In [11]:
from bitarray import bitarray
from time import sleep

### SHR
Shift right